Emily Yao, yaowemily@gmail.com, United States of America, University of Florida, Data Science

Problem description:
Persistence of drugs, otherwise defined as the duration between initial drug use and its discontinuation, is a big challenge for Pharmaceutical companies. However, with the use of machine learning, we can automate its identification by gathering insights on the factors impacting drug persistence.


In [11]:
import numpy as np
import pandas as pd
import plotly.express as px

In [12]:
df = pd.read_excel("https://github.com/qngyn/DataGlacier-Healthcare/blob/main/Data.xlsx?raw=true")

In [13]:
#@title Handling Outliers

#method one: completly removing them
def find_outliers(df):
   q1=df.quantile(0.25)
   q3=df.quantile(0.75)
   IQR=q3-q1
   outliers = df[((df<(q1-1.5*IQR)) | (df>(q3+1.5*IQR)))]
   return outliers
   
print("Finding outliers for 'Dexa_Freq_During_Rx'")
outliers = find_outliers(df['Dexa_Freq_During_Rx'])
df.drop(outliers, axis='index')

#Vizualizations
px.box(df, y= "Dexa_Freq_During_Rx")
px.box(df, y= "Count_Of_Risks")

df.skew(axis = 0, skipna = True) #reduced the skewness 

Finding outliers for 'Dexa_Freq_During_Rx'


<ipython-input-13-aa05f91c9a42>:19: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



Dexa_Freq_During_Rx    6.808730
Count_Of_Risks         0.879791
dtype: float64

In [14]:
#method two: Flooring & Capping
import statistics
Q1 = df['Dexa_Freq_During_Rx'].quantile(0.25)
Q3 = df['Dexa_Freq_During_Rx'].quantile(0.75)
IQR = Q3 - Q1
whisker_width = 1.5
lower_whisker = Q1 - (whisker_width*IQR)
upper_whisker = Q3 + (whisker_width*IQR)
df['Dexa_Freq_During_Rx'] = np.where(df['Dexa_Freq_During_Rx']>upper_whisker,upper_whisker,np.where(df['Dexa_Freq_During_Rx']<lower_whisker,lower_whisker,df['Dexa_Freq_During_Rx']))

Q1 = df['Count_Of_Risks'].quantile(0.25)
Q3 = df['Count_Of_Risks'].quantile(0.75)
IQR = Q3 - Q1
whisker_width = 1.5
lower_whisker = Q1 - (whisker_width*IQR)
upper_whisker = Q3 + (whisker_width*IQR)
df['Count_Of_Risks'] = np.where(df['Count_Of_Risks']>upper_whisker,upper_whisker,np.where(df['Count_Of_Risks']<lower_whisker,lower_whisker,df['Count_Of_Risks']))

#Vizualization
px.box(df, y= "Dexa_Freq_During_Rx")
px.box(df, y= "Count_Of_Risks")

df.skew(axis = 0, skipna = True) #reduced the skewness 

<ipython-input-14-55c5c8330c15>:23: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



Dexa_Freq_During_Rx    1.254572
Count_Of_Risks         0.778326
dtype: float64

In [15]:
#@title Skewness

I checked the skewness for the two different different methods of cleaning outliers and both of them reduced the skewness. The skewness of "Dexa_Freq_During_Rx" was reduced significantly more than "Count_Of_Risks". Furthermore, the second method was much more effective at handling skewness than the first.

In [16]:
#Vizualizations of new skewness
Rx_Skew = px.histogram(df, x='Dexa_Freq_During_Rx')
Rx_Skew.show()

Risks_Skew = px.histogram(df, x='Count_Of_Risks')
Risks_Skew.show()

A Note about ambigious indexing: 

They are found in these columns:

Age_Bucket

Tscore_Bucket_During_Rx

Tscore_Bucket_Prior_Ntm

There is no need to change them as they do cover a specific range, it would be appropriate to perform EDA on them as is and note any irregularities.

In [17]:
#@title Missing Information
#method one: deleting the rows of missing value
data_with_index = df.set_index("Risk_Segment_During_Rx")
data_with_index = data_with_index.drop("Unknown")

data_with_index2 = df.set_index("Tscore_Bucket_During_Rx")
data_with_index2 = data_with_index2.drop("Unknown")

data_with_index3 = df.set_index("Change_T_Score")
data_with_index3 = data_with_index3.drop("Unknown")

data_with_index4 = df.set_index("Change_Risk_Segment")
data_with_index4 = data_with_index4.drop("Unknown")
#this method greatly reduces the dataset so this is not preferred 

In [18]:
#method two: replacing missing value with the pervious input
df = df.replace('Unknown', np.NaN) 
df = df.fillna(method = 'bfill', limit = 1)
df['Change_Risk_Segment']
dfdrop = df.dropna()
dfdrop #significantly less data dropped, preferred method



,Ptid,Persistency_Flag,Gender,Race,Ethnicity,Region,Age_Bucket,Ntm_Speciality,Ntm_Specialist_Flag,Ntm_Speciality_Bucket,...,Risk_Family_History_Of_Osteoporosis,Risk_Low_Calcium_Intake,Risk_Vitamin_D_Insufficiency,Risk_Poor_Health_Frailty,Risk_Excessive_Thinness,Risk_Hysterectomy_Oophorectomy,Risk_Estrogen_Deficiency,Risk_Immobilization,Risk_Recurring_Falls,Count_Of_Risks
1,P2,Non-Persistent,Male,Asian,Not Hispanic,West,55-65,GENERAL PRACTITIONER,Others,OB/GYN/Others/PCP/Unknown,...,N,N,N,N,N,N,N,N,N,0.0
2,P3,Non-Persistent,Female,Other/Unknown,Hispanic,Midwest,65-75,GENERAL PRACTITIONER,Others,OB/GYN/Others/PCP/Unknown,...,N,Y,N,N,N,N,N,N,N,2.0
3,P4,Non-Persistent,Female,Caucasian,Not Hispanic,Midwest,>75,GENERAL PRACTITIONER,Others,OB/GYN/Others/PCP/Unknown,...,N,N,N,N,N,N,N,N,N,1.0
6,P7,Non-Persistent,Female,Caucasian,Not Hispanic,Midwest,>75,GENERAL PRACTITIONER,Others,OB/GYN/Others/PCP/Unknown,...,N,N,N,N,N,N,N,N,N,1.0
7,P8,Non-Persistent,Female,Caucasian,Not Hispanic,Midwest,>75,GENERAL PRACTITIONER,Others,OB/GYN/Others/PCP/Unknown,...,N,N,N,N,N,N,N,N,N,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3410,P3411,Non-Persistent,Male,Caucasian,Not Hispanic,South,65-75,GENERAL PRACTITIONER,Others,OB/GYN/Others/PCP/Unknown,...,N,N,Y,N,N,N,N,N,N,1.0
3415,P3416,Non-Persistent,Female,Caucasian,Not Hispanic,South,>75,ORTHOPEDICS,Others,OB/GYN/Others/PCP/Unknown,...,N,N,N,Y,N,N,N,N,N,1.0
3416,P3417,Persistent,Female,Caucasian,Not Hispanic,South,>75,GENERAL PRACTITIONER,Others,OB/GYN/Others/PCP/Unknown,...,N,N,N,N,N,N,N,N,N,2.0
3420,P3421,Persistent,Female,Caucasian,Not Hispanic,South,>75,ENDOCRINOLOGY,Others,OB/GYN/Others/PCP/Unknown,...,N,N,N,N,N,N,N,N,N,0.0


Notes: no duplicate values, missing values, date-parsing issues to consider with our dataset